In [1]:
!pip install opendatasets -q
import opendatasets as od
link = 'https://www.kaggle.com/datasets/sakshamjn/vehicle-detection-8-classes-object-detection'
od.download(link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ravirajkukade
Your Kaggle Key: ··········


100%|██████████| 346M/346M [00:04<00:00, 83.4MB/s]

In [2]:
%cd /content/vehicle-detection-8-classes-object-detection/train
!ls -la

/content/vehicle-detection-8-classes-object-detection/train
total 2492
drwxr-xr-x 4 root root   4096 Oct  6 20:30 .
drwxr-xr-x 3 root root   4096 Oct  6 20:30 ..
drwxr-xr-x 2 root root 786432 Oct  6 20:30 images
drwxr-xr-x 2 root root 774144 Oct  6 20:30 labels
-rw-r--r-- 1 root root 980955 Oct  6 20:30 labels.npy


In [3]:
import os
import random
import shutil

# Set up source and destination paths
source_folder = './'
train_folder = './traindata'
test_folder = './testdata'
valid_folder = './validdata'

# Create destination folders
for folder in [train_folder, test_folder, valid_folder]:
    os.makedirs(os.path.join(folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(folder, 'labels'), exist_ok=True)

# Get list of image files
image_files = [filename for filename in os.listdir(os.path.join(source_folder, 'images')) if filename.endswith('.jpg')]

# Randomize the order of the image files
random.shuffle(image_files)

# Determine split sizes
train_size = int(0.7 * len(image_files))
test_size = int(0.15 * len(image_files))
valid_size = len(image_files) - train_size - test_size

# Copy images and labels to the destination folders
for i, filename in enumerate(image_files):
    image_path = os.path.join(source_folder, 'images', filename)
    label_path = os.path.join(source_folder, 'labels', filename.replace('.jpg', '.txt'))

    if i < train_size:
        destination_folder = train_folder
    elif i < train_size + test_size:
        destination_folder = test_folder
    else:
        destination_folder = valid_folder

    shutil.copy(image_path, os.path.join(destination_folder, 'images'))
    shutil.copy(label_path, os.path.join(destination_folder, 'labels'))

In [4]:
!pip install ultralytics
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from IPython.core.magic import register_line_cell_magic
import yaml
from PIL import Image
import os
import seaborn as sns
import ultralytics
from ultralytics import YOLO
from matplotlib.patches import Rectangle
import glob
import cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 kB 4.0 MB/s eta 0:00:00


In [7]:
import os
from PIL import Image

# Set up folder paths
train_folder = './traindata/images'
test_folder = './testdata/images'
valid_folder = './validdata/images'
target_size = (224, 224)  # Define the desired size for the images

# Function to resize images in a folder
def resize_images_in_folder(folder_path):
    # Get list of image files
    image_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

    # Resize images in the folder
    for filename in image_files:
        image_path = os.path.join(folder_path, filename)

        # Open the image using PIL
        image = Image.open(image_path)

        # Resize the image to the target size
        resized_image = image.resize(target_size)

        # Save the resized image
        resized_image.save(image_path)

# Resize images in the train folder
resize_images_in_folder(train_folder)

# Resize images in the test folder
resize_images_in_folder(test_folder)

# Resize images in the valid folder
resize_images_in_folder(valid_folder)

In [8]:
for mode in ['traindata', 'validdata', 'testdata']:
    print(f'\nImage sizes in {mode} set:\n')
    img_size = 0
    for file in glob.glob(os.path.join('./', mode, 'images', '*')):
        image = Image.open(file)
        if image.size != img_size:
            #print(f'\t{image.size}')
            img_size = image.size


Image sizes in traindata set:


Image sizes in validdata set:


Image sizes in testdata set:



In [9]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 76.2MB/s]


In [10]:
num_classes = 8
classes = ['Motorcycle','Auto','Car','Bus','LCV(Light Motor Vehicle)','Truck','Tractor','Multi-Axle']


dict_file = {'train': os.path.join('./', 'traindata'),
             'val': os.path.join('./', 'validdata'),
             'test': os.path.join('./', 'testdata'),
             'nc': num_classes,
             'names': classes
            }

with open(os.path.join('./', 'vehicledata.yaml'), 'w+') as file:
    yaml.dump(dict_file, file)

In [11]:

# define number of classes based on YAML
import yaml
with open("./vehicledata.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

num_classes

'8'

In [15]:
model.train(data= '/content/vehicle-detection-8-classes-object-detection/train/vehicledata.yaml',
            task='detect',
            imgsz=224,
            epochs=100,
            batch=32,
            mode='train',
            name='yolov8n_v1_train')


Ultralytics YOLOv8.0.194 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/vehicle-detection-8-classes-object-detection/train/vehicledata.yaml, epochs=100, patience=50, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v1_train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a94dc795ab0>
fitness: 0.5730984262169148
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.46166,     0.63024,     0.62425,     0.51815,     0.41371,     0.59258,     0.59606,     0.54807])
names: {0: 'Motorcycle', 1: 'Auto', 2: 'Car', 3: 'Bus', 4: 'LCV(Light Motor Vehicle)', 5: 'Truck', 6: 'Tractor', 7: 'Multi-Axle'}
plot: True
results_dict: {'metrics/precision(B)': 0.7735425462147396, 'metrics/recall(B)': 0.7180847148162436, 'metrics/mAP50(B)': 0.798179031001608, 'metrics/mAP50-95(B)': 0.5480894701297265, 'fitness': 0.5730984262169148}
save_dir: PosixPath('runs/detect/yolov8n_v1_train4')
speed: {'preprocess': 0.13169303319056186, 'inference': 0.4747441367546589, 'loss': 0.0007115847867350525, 'po

In [19]:
model = YOLO('./runs/detect/yolov8n_v1_train4/weights/best.pt')

In [20]:
results = model.predict(source=os.path.join('./', 'testdata', 'images'), save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1232 /content/vehicle-detection-8-classes-object-detection/train/testdata/images/Highway_1006_2020-07-30_jpg.rf.c58b7b2098d969e925b271bcd436ac25.jpg: 224x224 4 Cars, 10.8ms
image 2/1232 /content/vehicle-detection-8-classes-object-detection/train/testdata/images/Highway_1007_2020-07-30_jpg.rf.488f26a3ee04a30ac422151d4a2b1935.jpg: 224x224 1 Motorcycle, 1 Car, 4 LCV(Light Motor Vehicle)s, 10.8ms
image 3/1232 /content/vehicle-detection-8-classes-ob

In [21]:
%matplotlib inline

predicitions = glob.glob(os.path.join('./', 'runs/detect/predict', '*'))

n = 10

for i in range(n):
    idx = np.random.randint(0, len(predicitions))
    image = Image.open(predicitions[idx])
    plt.imshow(image)
    plt.grid(False)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [22]:
import cv2

# Load the video
video_path = "/content/4K Video of Highway Traffic!.mkv"  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Get the video frame width and height
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to save the output video
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'XVID'), 10, (frame_width, frame_height))

# Process each frame in the video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    # Draw bounding boxes on the frame
    annotated_frame = results.render()[0]

    # Display the frame with bounding boxes
    cv2.imshow('Frame', annotated_frame)

    # Write the frame with bounding boxes to the output video
    out.write(annotated_frame)

    # Press 'q' to exit the video playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
out.release()

# Destroy OpenCV windows
cv2.destroyAllWindows()



0: 128x224 (no detections), 103.6ms
Speed: 3.0ms preprocess, 103.6ms inference, 1.7ms postprocess per image at shape (1, 3, 128, 224)


AttributeError: ignored